<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [91]:
from collections import Counter
import random
#from string import ascii_uppercase

In [92]:
class Board:
    """
    <Doc string>
    """
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.slots = {(i, j): 'pick me!' for i in range(height) for j in range(width)}

In [93]:
class Card:
    """
    """
    def __init__(self, word):
        self.word = word
        self.flip = False

In [94]:
class Player:
    def __init__(self, player_number):
        self.record = None
        if player_number == 1:
            self.name = input('Please input name for Player {} (Default is Mario): '.format(player_number)) or 'Mario'
            self.turn = True
        elif player_number == 2:
            self.name = input('Please input name for Player {} (Default is Luigi): '.format(player_number)) or 'Luigi'
            self.turn = False

In [95]:
class Game:
    """
    
    Input(integer):
    """
    def __init__(self, selection):
        if selection == 1:
            self.board = Board(6, 3)
        elif selection == 2:
            self.board = Board(9, 4)
        else:
            print("Please ...")  #Error message???
        self.gamestate = self.board.slots
        self.flame_count = 0
        self.ice_count = 0
        self.flight_count = 0
        self.powerup_count = 0
        self.bowser_count = 0
        self.bowserjr_count = 0
        self.exit = False
        
    def create_deck(self, size, r_to_p_ratio):
        """
        Input:
            size(integer): size of deck
            r_to_p_ratio(float): reward to penalty ratio in the deck
        Output:
            deck(list): a list consisting the contents of each card in the deck
        """
        self.penalty = int(size / (r_to_p_ratio + 1))
        self.rewards = size - self.penalty
        rewards = Counter({'flame'   : self.rewards*2//7,
                           'ice'     : self.rewards*2//7,
                           'flight'  : self.rewards*2//7,
                           'powerup' : self.rewards*1//7})
        penalties = Counter({'bowser'  : self.penalty*1//2,
                             'bowserjr': self.penalty*1//2})
        all_items = rewards + penalties
        self.dead_by_bowser = int(penalties['bowser'])
        self.dead_by_bowserjr = int(penalties['bowserjr'])
        deck = []
        for key, value in all_items.items():
            for i in range(value):
                deck.append(Card(str(key)))
        return deck
    
    def shuffle_deck(self, deck):
        random.shuffle(deck)
        return deck
    
    def place_deck(self, deck, board):
        board_with_cards={}
        keys = list(board.slots.keys())
        for count, key in enumerate(keys):
            board_with_cards[key] = deck[count]
        return board_with_cards
    
    def flip_card_in_game(self, gs, pos):
        gs[pos].flip = True
        self.update_gamestate(gs, pos)
        return gs
        
    def update_gamestate(self, gs, pos):
        card = gs[pos]
        if card.flip:
            self.gamestate[pos] = card.word
            self.update_rewards(card.word)
            self.update_bowsers(card.word)
        else:
            self.gamestate[pos] = 'pick me!'
        
                
    def print_board(self):
        divider = '  +========+========+========+========+========+========+'
        b = []
        b.append('      0        1        2        3        4        5     ')
        b.append(divider)
        for i in range(mygame.board.width):
            a = str(i) + ' |'
            for j in range(mygame.board.height):
                a = a + mygame.gamestate[(j,i)].ljust(8) + '|'
            b.append(a)
            b.append(divider)
        for m in range(len(b)):
            print(b[m])
#         print('      0        1        2        3        4        5     ')
#         print('  +========+========+========+========+========+========+') # use join
# #         for key in self.board.slots.keys():
# #             str0 = ''.join(self.gamestate[key])
# #         print(str0)
#         print('0 |{:8}|{:8}|{:8}|{:8}|{:8}|{:8}|'.format(self.gamestate[(0,0)], self.gamestate[(1,0)], self.gamestate[(2,0)], self.gamestate[(3,0)], self.gamestate[(4,0)], self.gamestate[(5,0)]))
#         print('  +========+========+========+========+========+========+')
#         print('1 |{:8}|{:8}|{:8}|{:8}|{:8}|{:8}|'.format(self.gamestate[(0,1)], self.gamestate[(1,1)], self.gamestate[(2,1)], self.gamestate[(3,1)], self.gamestate[(4,1)], self.gamestate[(5,1)]))
#         print('  +========+========+========+========+========+========+')
#         print('2 |{:8}|{:8}|{:8}|{:8}|{:8}|{:8}|'.format(self.gamestate[(0,2)], self.gamestate[(1,2)], self.gamestate[(2,2)], self.gamestate[(3,2)], self.gamestate[(4,2)], self.gamestate[(5,2)]))
#         print('  +========+========+========+========+========+========+')
    
    def players_list(self):
        self.players = []
        for i in range(1,3):
            self.players.append(Player(i))
    
    def find_current_player(self):
        for p in self.players:
            if p.turn:
                return p
            
    def change_current_player(self):
        for p in self.players:
            if p.turn:
                p.turn = False
            else:
                p.turn = True
    
#     def ASCII_to_int(self, upper_ascii):
#         ascii_to_int_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5}
#         return ascii_to_int_dict[upper_ascii]

#     def input_to_coord(self, player_input):
        
        
    def ask_for_pick(self, gs):
        print('{}, Please pick a card!'.format(self.find_current_player().name))
        while True:
            if self.exit:
                break
            while True:
                x_input = input("Please pick a card (enter coordinate x): ")
                if x_input == "exit":
                    self.exit = True
                    break
                elif x_input == "rules":
                    self.print_rules()
                    continue
                try:
                    x = int(x_input)
                except ValueError:
                    print("{} is not a valid number, please try again".format(x_input))
                    continue
                if not (-1 < x < self.board.height):
                    print("Please enter a value between 0 and {}".format(self.board.height-1))
                    continue
                else:
                    break
            if not self.exit:
                while True:
                    y_input = input("Please pick a card (enter coordinate y): ")
                    if y_input == "exit":
                        self.exit = True
                        break
                    elif y_input == "rules":
                        self.print_rules()
                        continue
                    try:
                        y = int(y_input)
                    except ValueError:
                        print("{} is not a valid number, please try again".format(y_input))
                        continue
                    if not (-1 < y < self.board.width):
                        print("Please enter a value between 0 and {}".format(self.board.width-1))
                        continue
                    else:
                        break
            if not self.exit:
                if gs[(x,y)].flip:
                    print("Card already picked, please pick again.")
                    continue
                else:
                    return (x,y)

    def display_pick(self, gs, pick):
        print('{} picked {}, it revealed the card [{}]'.format(self.find_current_player().name, pick, gs[pick].word))
            
    def update_rewards(self, word):
        if word == 'flame':
            self.flame_count += 1
        elif word == 'ice':
            self.ice_count += 1
        elif word == 'flight':
            self.flight_count += 1
        elif word == 'powerup':
            self.powerup_count += 1
            
    def update_bowsers(self, word):
        if word == 'bowser':
            self.bowser_count += 1
        elif word == 'bowserjr':
            self.bowserjr_count += 1
        
    def print_rules(self):
        print('Game Rules')
        
    def print_current_count(self):
        print('Current card counts are:')
        print('{}x Flame  Powerup Card(s) ==> {}x Flame  Powerup(s)'.format(self.flame_count, self.flame_count//2))
        print('{}x Ice    Powerup Card(s) ==> {}x Ice    Powerup(s)'.format(self.ice_count, self.ice_count//2))
        print('{}x Flight Powerup Card(s) ==> {}x Flight Powerup(s)'.format(self.flight_count, self.flight_count//2))
        print('{}x Normal Powerup Card(s) ==> {}x Normal Powerup(s)'.format(self.powerup_count, self.powerup_count//2))
        print('{}x Bowser         Card(s)'.format(self.bowser_count))
        print('{}x Bowser Jr.     Card(s)'.format(self.bowserjr_count))
        
    def print_results(self):
        print('Game Over!')
        if self.bowser_count >=2:
            print('You were kicked out by Bowser!')
        elif self.bowserjr_count >=2:
            print('You were kicked out by Bowser Jr.!')
        elif self.exit:
            print('Goodbye! See you next time!')
        else:
            print('Amazing! You gained all the rewards!')
        print('You have gained:') # use join
        print('{}x Flame  Powerup(s)'.format(self.flame_count//2))
        print('{}x Ice    Powerup(s)'.format(self.ice_count//2))
        print('{}x Flight Powerup(s)'.format(self.flight_count//2))
        print('{}x Normal Powerup(s)'.format(self.powerup_count//2))

In [96]:
# mygame = Game(1)
# mygame.ASCII_to_int('A')

In [97]:
# def input_to_coord(player_input):
#     x = player_input[0]
#     y = int(player_input[1])
#     return (x,y)

In [98]:
# divider = '  +========+========+========+========+========+========+'
# b = []
# #b.append('      0        1        2        3        4        5     ')
# b.append('      A        B        C        D        E        F     ')
# b.append(divider)
# for i in range(mygame.board.width):
#     a = str(i) + ' |'
#     for j in range(mygame.board.height):
#         a = a + mygame.gamestate[(j,i)].ljust(8) + '|'
#     b.append(a)
#     b.append(divider)
    
# for m in range(len(b)):
#     print(b[m])

In [99]:
#def main(board_size_selection, r_to_p_ratio):
mygame = Game(1)
mydeck = mygame.create_deck(len(mygame.board.slots), 3.5)
mydeck_shuffled = mygame.shuffle_deck(mydeck)
gs = mygame.place_deck(mydeck_shuffled,mygame.board)
mygame.players_list()
mygame.print_board()
while mygame.bowser_count < mygame.dead_by_bowser and mygame.bowserjr_count < mygame.dead_by_bowserjr and not mygame.exit:
    pick = mygame.ask_for_pick(gs)
    if not mygame.exit:
        mygame.display_pick(gs, pick)
        mygame.change_current_player()
        mygame.flip_card_in_game(gs, pick)
        mygame.print_board()
        mygame.print_current_count()
mygame.print_results()

Please input name for Player 1 (Default is Mario): 
Please input name for Player 2 (Default is Luigi): 
      0        1        2        3        4        5     
  +========+========+========+========+========+========+
0 |pick me!|pick me!|pick me!|pick me!|pick me!|pick me!|
  +========+========+========+========+========+========+
1 |pick me!|pick me!|pick me!|pick me!|pick me!|pick me!|
  +========+========+========+========+========+========+
2 |pick me!|pick me!|pick me!|pick me!|pick me!|pick me!|
  +========+========+========+========+========+========+
Mario, Please pick a card!
Please pick a card (enter coordinate x): 0
Please pick a card (enter coordinate y): 0
Mario picked (0, 0), it revealed the card [powerup]
      0        1        2        3        4        5     
  +========+========+========+========+========+========+
0 |powerup |pick me!|pick me!|pick me!|pick me!|pick me!|
  +========+========+========+========+========+========+
1 |pick me!|pick me!|pick me!|pick 

In [ ]:
input('Please input name for Player {}: '.format(player_number)) or 'Mario' if player_number == 1:

In [ ]:
#Main

#initiate game with size of board

#Fill board with cards

#Player 1 pick one card by position

#Player 2 pick one card

#Continues until dead or all rewards gained

